# REST and Python: Tools of the Trade:
- There are 3 popular frameworks in Python that can be used for building REST APIs in Python.
- All the examples will be for a similar API that manages a collection of countries.
- Each country will have the following fields:
  - ``name`` is the name of the country.
  - ``capital`` is the capital of the country.
  - ``area`` is the area of the country in square kilometers.
- The fields name, capital, and area store data about a specific country somewhere in the world.
- For the examples below, you’ll store your data in a Python list. However, in the real world, this data is stored in a Database.
- The exception to this is the Django REST framework example, which runs off the *SQLite* database that Django creates.
- To keep things consistent, we use ``countries`` as the main endpoint for all three frameworks. We also use JSON as the data format for all three frameworks.

## 1. Flask:
- It is a Python microframework used to build web apps and REST APIs.
- Flask provides a solid backbone for your applications while leaving many design choices up to you. 
- Flask’s main job is to handle HTTP requests and route them to the appropriate function in the application.
---
**Note**

- The code in this section uses the new Flask 2 syntax. 
- If you’re running an older version of Flask, then use ``@app.route("/countries")`` instead of ``@app.get("/countries")`` and ``@app.post("/countries")``.

- To handle POST requests in older versions of Flask, you’ll also need to add the methods parameter to ``@app.route()``:

  ``@app.route("/countries", methods=["POST"])``
- This route handles POST requests to ``/countries`` in Flask 1.

---
- Below is an example Flask application for the REST API:

```python
# app.py
from flask import Flask, request, jsonify

app = Flask(__name__)

countries = [
    {"id": 1, "name": "Thailand", "capital": "Bangkok", "area": 513120},
    {"id": 2, "name": "Australia", "capital": "Canberra", "area": 7617930},
    {"id": 3, "name": "Egypt", "capital": "Cairo", "area": 1010408},
]

def _find_next_id():
    return max(country["id"] for country in countries) + 1

@app.get("/countries")
def get_countries():
    return jsonify(countries)

@app.post("/countries")
def add_country():
    if request.is_json:
        country = request.get_json()
        country["id"] = _find_next_id()
        countries.append(country)
        return country, 201
    return {"error": "Request must be JSON"}, 415
```
- This application defines the API endpoint ``/countries`` to manage the list of countries. It handles two different kinds of requests:
  1. GET ``/countries`` returns the list of countries.
  2. POST ``/countries`` adds a new country to the list.
- Before running this application, we would need to install ``flask`` with pip.
```shell
python -m pip install flask
```
- Once ``flask`` is installed, save the code in a file called ``app.py``. To run this Flask application, you first need to set an environment variable called ``FLASK_APP`` to app.py. This tells Flask which file contains your application.
- Run the following command inside the folder that contains app.py:
```shell
export FLASK_APP=app.py
```
- This sets ``FLASK_APP`` to ``app.py`` in the current shell. Optionally, you can set ``FLASK_ENV`` to ``development``, which puts Flask in **debug** mode. Without debug mode, you’d have to restart the server after every change.
```shell
export FLASK_ENV=development
```
---
**Note:**
- The above commands work on macOS or Linux. If you’re running this on Windows, then you need to set FLASK_APP and FLASK_ENV like this in the Command Prompt:
```Windows Command Prompt
C:\> set FLASK_APP=app.py
C:\> set FLASK_ENV=development
```
- Now FLASK_APP and FLASK_ENV are set inside the Windows shell.
---
- With all the environment variables ready, you can now start the Flask development server by calling flask run:
```
flask run
* Serving Flask app "app.py" (lazy loading)
* Environment: development
* Debug mode: on
* Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
```
- This starts up a server running the application. Open up your browser and go to http://127.0.0.1:5000/countries, and you’ll see the following response:
```json
[
    {
        "area": 513120,
        "capital": "Bangkok",
        "id": 1,
        "name": "Thailand"
    },
    {
        "area": 7617930,
        "capital": "Canberra",
        "id": 2,
        "name": "Australia"
    },
    {
        "area": 1010408,
        "capital": "Cairo",
        "id": 3,
        "name": "Egypt"
    }
]
```
- This JSON response contains the three countries defined at the start of ``app.py``.

### Let's have a code walkthrough:
#### 1. get_countries():

```python
@app.get("/countries")
def get_countries():
    return jsonify(countries)
```

- ```@app.get()``` is a Flask decorator to connect GET requests to a function in the application.
- When the ``/countries`` is accessed, Flask calls this decorated function to handle the HTTP request and return a response.
- Also, ``get_countries()`` is a function which takes in the list ``countries`` and converts it to JSON with ``jsonify()``.
- **Note**: In most of the cases, Flask automatically converts Python dictionary to JSON, however, this is only supported for **one single dictionary** at a time and not a list of dictionaries (as in case of our example):
```python
@app.get("/country")
def get_country():
    return countries[1]
```
- In this case, we return the second dictionary from countries which is automatically converted to JSON by Flask and the response is as below:
```json
{
    "area": 7617930,
    "capital": "Canberra",
    "id": 2,
    "name": "Australia"
}
```
- In ``get_countries()``, you need to use ``jsonify()`` because you’re returning a **list of dictionaries** and not just a single dictionary. Flask doesn’t automatically convert lists to JSON!

#### 2. add_country():
```python
@app.post("/countries")
def add_country():
    if request.is_json():
        country = request.get_json()
        # Adding the id for the new country
        country["id"] = _find_next_id()
        countries.append(country)
        return country, 201                          # Created
    return {"error": "Request must be JSON"}, 415    # Unsupported media type
```
- This function performs the following operations:
    1. Checks if request is JSON -- ``request.is_json()``
    2. If True, creates a new ``country`` instance -- ``request.get_json()``
    3. Finds the next id to be set for this country and sets it in the JSON -- ``_find_next_id()``
    4. Appends the new ``country`` object to ``countries`` -- ``countries.append(country)``
    5. Returns the country in response along with the ``201 Created`` status code.
    6. If False, Returns an error with ``415 Unsupported Media Type`` status code in case if the media type was not JSON.
- ``add_country()`` also calls ``_find_next_id()`` to determine the id for the new country:
```python
def _find_next_id():
    return max(country["id"] for country in countries) + 1
```
- This ``helper`` function uses a *generator* expression to select all the country IDs and then calls ``max()`` on them to get the largest value. It increments this value by 1 to get the next ID to use.

### Trying it out:
- We can use ``curl`` to try our endpoints in the shell.
- ``curl`` allows us to send HTTP requests from the commmand line.
```sh
curl -i http://127.0.0.1:5000/countries \
-X POST \
-H 'Content-Type: application.json' \
-d {"name":"Germany", "capital": "Berlin", "area": 357022}
```
- Below options are used here:
    - `-X`: sets the HTTP method for the request
    - `-H`: adds the HTTP header to the request
    - `-d`: defines the request data
- With these options, curl sends JSON data in a ``POST`` request with the ``Content-Type`` header set to ``application/json``. The REST API returns ``201 CREATED`` along with the JSON for the new country you added:
```json
HTTP/1.0 201 CREATED
Content-Type: application/json
...

{
    "area": 357022,
    "capital": "Berlin",
    "id": 4,
    "name": "Germany"
}
```
- **Note:** In this example, ``add_country()`` doesn’t contain any validation to confirm that the JSON in the request matches the format of countries. Check out [flask-expects-json](https://pypi.org/project/flask-expects-json/) if you’d like to validate the format of JSON in Flask.
- We can also use ``curl`` to send a ``GET`` request to ``/countries`` and confirm if the new country was added.
- If we don't use the ``-x`` option in command line with ``curl``, it sends a ``GET`` request by default:
```sh
curl -i http://127.0.0.1:5000/countries
```
- This returns the full list of countries in the system, with the newest country at the bottom:
```json
HTTP/1.0 200 OK
Content-Type: application/json
...

[
    {
        "area": 513120,
        "capital": "Bangkok",
        "id": 1,
        "name": "Thailand"
    },
    {
        "area": 7617930,
        "capital": "Canberra",
        "id": 2,
        "name": "Australia"
    },
    {
        "area": 1010408,
        "capital": "Cairo",
        "id": 3,
        "name": "Egypt"
    },
    {
        "area": 357022,
        "capital": "Berlin",
        "id": 4,
        "name": "Germany"
    }
]
```
- This is just a sampling of what Flask can do. 
- This application could be expanded to include endpoints for all the other HTTP methods. 
- Flask also has a large ecosystem of extensions that provide additional functionality for REST APIs, such as [database integrations](https://realpython.com/flask-connexion-rest-api/), [authentication](https://realpython.com/flask-google-login/), and background processing.

--------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------